# INE

El [Instituto Nacional de Estadística](https://www.ine.es/) publíca más de 5000 conjuntos de datos. 
Acceder a ellos desde la web oficial puede ser muy ineficiente. 

En este notebook, vamos a explorar como exportar todos los datos a un formato más eficiente (Parquet).

In [1]:
import httpx


def ine_request(endpoint):
    """Fetch data from INE API endpoint with automatic pagination."""
    page = 1
    data = []

    with httpx.Client(
        base_url="https://servicios.ine.es/wstempus/js/ES",
        limits=httpx.Limits(max_keepalive_connections=16),
        transport=httpx.HTTPTransport(retries=5),
    ) as client:
        while True:
            response = client.get(
                f"/{endpoint}",
                params={"det": 10, "page": page},
                follow_redirects=True,
                timeout=120,
            ).json()

            if not response:
                break

            data.extend(response)

            if len(response) < 500:
                break

            page += 1

    return data


In [2]:
# Obtenemos todas las operaciones disponibles
operaciones = ine_request("OPERACIONES_DISPONIBLES")

# Obtenemos todas las tablas
tablas = [
    tabla
    for operation in operaciones
    for tabla in ine_request(f"TABLAS_OPERACION/{operation['Id']}")
]

In [3]:
# Crear un nuevo directorio para los datos del INE
from pathlib import Path

# Crear el directorio 'ine' si no existe
ine_dir = Path("ine")
ine_dir.mkdir(exist_ok=True, parents=True)

print(f"Directorio creado: {ine_dir.absolute()}")

Directorio creado: /home/david/projects/datania/notebooks/ine


In [4]:
import json


def save_jsonl(data, filename):
    with open(filename, "w", encoding="utf-8") as f:
        for item in data:
            f.write(json.dumps(item, ensure_ascii=False) + "\n")


save_jsonl(operaciones, ine_dir / "operaciones.jsonl")
save_jsonl(tablas, ine_dir / "tablas.jsonl")


In [5]:
!ls -lh ine/*.jsonl

-rw-r--r-- 1 david david  29K Mar 17 17:12 ine/operaciones.jsonl
-rw-r--r-- 1 david david 7.0M Mar 17 17:12 ine/tablas.jsonl


In [ ]:
import duckdb

c = duckdb.connect()
c.sql("""
CREATE SECRET http (TYPE http, EXTRA_HTTP_HEADERS MAP {'Accept-Encoding': 'gzip'});
""")

failed_tables = []
total_tables = len(tablas)

for i, table in enumerate(tablas):
    # Si el directorio no existe, lo creamos
    table_dir = ine_dir / "tablas" / str(table["Id"])
    table_dir.mkdir(exist_ok=True, parents=True)

    # Copiamos el CSV a Parquet
    try:
        c.sql(f"""
            copy (
                from read_csv(
                    'https://www.ine.es/jaxiT3/files/t/en/csv_bdsc/{table["Id"]}.csv',
                    delim=';',
                    ignore_errors=true,
                    normalize_names=true,
                    null_padding=true,
                    parallel=true,
                    strict_mode=false,
                    compression='gzip'
                )
            )
            to 'ine/tablas/{table["Id"]}/datos.parquet' (
                format parquet,
                compression 'zstd',
                parquet_version v2,
                row_group_size 1048576
            );
        """)
    except Exception as e:
        print(f"Error processing table {table['Id']}: {str(e)}")
        failed_tables.append(table["Id"])

Processing table 1/5030: 10275
Processing table 51/5030: 69317
Processing table 101/5030: 59973
Processing table 151/5030: 2910
Error processing table 2864: Query interrupted


In [ ]:
if failed_tables:
    print(f"\nFailed to process {len(failed_tables)} tables:")
    for table_id in failed_tables:
        print(f"- {table_id}")
else:
    print("\nAll tables processed successfully!")
